**тематическое моделирование**

Евгений Борисов borisov.e@solarl.ru

In [1]:
# http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

# разложение частотной матрицы [ слова x документы ]  
#
# получаем матрицу с описанием тем [ слова х темы ]   
# и матрицу вероятностей событий "тема описывает документ"  [ темы х документы ]
# 
# [ слова x документы ] = [ слова х темы ] * [ темы х документы ]
# 
# p(w|d) = p(w|t) * p(t|d)

In [2]:
import sys
import re
import gzip
import pandas as pd

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [4]:
pd.options.display.max_colwidth = 200  

In [5]:
n_features = 1000
n_components = 10

df = pd.read_pickle('../data/text/news.pkl.gz')
print('текстов:',len(df))
df.sample(4)

текстов: 3196


,text,tag
1324,"Обломок российского ""Прогресса"" нашли во дворе дома\n\n5 декабря 2016 в 12:18\n\n42.TUT.BY\n\nОбломок космического грузового корабля «Прогресс» найден во дворе жилого дома. Об этом сообщает официа...",tech
2320,"Как установлено следствием, погибшие мужчина и женщина являются 46-летней супружеской парой местных жителей.\n\nСледственным отделом по городу Серпухов ГСУ СК России по Московской области в отноше...",incident
111,"""Локомотив"" одержал волевую победу над ""Брондбю"" Первый тайм железнодорожники\nпровели крайне слабо, как и ряд последних игр чемпионата страны. Две\nгрубейшие ошибки в центре обороны в первом тайм...",sport
568,В центре Бреста спасатели вынесли из горящей квартиры двух человек\n\n2 декабря 2016 в 10:23\n\nTUT.BY\n\nВо время пожара в многоквартирном доме на проспекте Машерова в Бресте сотрудники МЧС спасл...,incident


In [6]:
data = df['text'].tolist()

In [7]:
with gzip.open('../data/text/stop-nltk.txt.gz','rt',encoding='utf-8') as f: 
    stopwords = set([ w.strip() for w in  f.read().split() if w.strip() ] )

print('количество стоп-слов:',len(stopwords))

sorted(stopwords)

количество стоп-слов: 151


['а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [8]:
%%time 

df['text_clean'] = df['text'].str.lower() # приведение в lowercase

# замена символов-разделителей (-,_) на пробел
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'_', ' ', s))

# замена цифр
df['text_clean'] = df['text_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' ', s))

# делим строки на слова
df['text_clean'] = df['text_clean'].apply(lambda t: [ w.strip() for w in t.split() if len(w.strip())>2 ] )

# удаление лишних слов
df['text_clean'] = df['text_clean'].apply(lambda t:[w for w in t if w not in stopwords])

CPU times: user 573 ms, sys: 23.1 ms, total: 596 ms
Wall time: 597 ms


In [9]:
df[['text_clean']].sample(10)

,text_clean
45,"[британские, ученые, доказали, плохой, учебе, виноват, рост, терапии, использованием, факторов, роста, отмечалось, ускорение, роста, улучшение, следовательно, интеллектуальные, способности, также,..."
2194,"[время, пожара, складе, пиломатериалов, москве, обрушились, кровля, стены, здания, тушении, горящего, одноэтажного, деревянного, строения, никто, пострадал, сообщает, корреспондент, мир, настоящее..."
3012,"[российские, автомобильные, эксперты, составили, рейтинг, седанов, которые, году, покинули, автомобильный, рынок, россии, отмечается, часть, автомобилей, снята, продажи, низкого, спроса, российско..."
2940,"[сообщается, причиной, данного, решения, является, повышение, конкуренции, стороны, современных, переднеприводных, автомоделей, строительство, транспортных, средствна, новой, платформе, позволит, ..."
1122,"[штаб, сборной, беларуси, биатлону, назвал, составы, этап, кубка, мира, поклюке, декабря, sport, tut, анна, сола, анастасия, киннунен, фото, deubert, тренерский, штаб, сборной, беларуси, биатлону,..."
2881,"[автопроизводители, audi, mercedes, benz, volvo, отзывают, около, тыс, автомобилей, проблем, подушками, ремнями, безопасности, фарами, сообщает, yonhap, ссылкой, министерство, транспорта, южной, к..."
1991,"[тренировочный, сбор, александра, поветкина, проходил, спортивной, базе, отдыха, чехове, московская, область, летний, александр, поветкин, боксёр, профессионал, выступающий, тяжёлой, весовой, кате..."
62,"[курение, кальяна, оказаться, вреднее, обычное, курение, курение, кальяна, зародившееся, несколько, столетий, назад, ближнем, востоке, стремительно, входит, моду, странах, европы, сша, молодых, лю..."
2896,"[корейская, компания, kia, опубликовала, второй, тизер, нового, спорткара, ожидается, публичная, премьера, состоится, начале, года, свежем, видеоролике, увидеть, новый, спортивный, автомобиль, про..."
200,"[беларуси, будут, сажать, подготовку, участников, акций, протеста, президент, беларуси, александр, лукашенко, инициирует, введение, уголовной, ответственности, действия, направленные, дискредитаци..."


In [10]:
# sorted(set([ w for t in  df['text_clean'] for w in t ]))

In [11]:
# собираем слова в строку
df['text_clean'] = df['text_clean'].apply(lambda t: ' '.join(t) )

In [12]:
%xdel stopwords

In [13]:
df[ df['text_clean'].str.len()<1 ]

,text,tag,text_clean


In [14]:
df.sample(10)

,text,tag,text_clean
2178,"Днём 13 декабря в Кузбассе на 106 километре трассы «Ленинск-Кузнецкий – Новокузнецк» произошло шесть ДТП с участием девяти машин.\n\nПредварительно, сначала на этом участке дороги произошло нескол...",incident,днём декабря кузбассе километре трассы ленинск кузнецкий новокузнецк произошло шесть дтп участием девяти машин предварительно сначала участке дороги произошло несколько аварий которых повреждения ...
165,"Кубок мира по шахматам. Двое россиян начали IV раунд с побед Александр\nГрищук взял верх над американцем Гата Камским (Gata Kamsky), а Сергей\nРублевский нанес поражение своему соотечественнику Ко...",sport,кубок мира шахматам двое россиян начали раунд побед александр грищук взял верх американцем гата камским gata kamsky сергей рублевский нанес поражение своему соотечественнику константину сакаеву ук...
478,Цена барреля нефти Brent превысила отметку в 55 долларов впервые с июля 2015 года\n\n5 декабря 2016 в 13:40\n\nТАСС\n\nЦена на нефть марки Brent на бирже ICE в Лондоне выросла на 1%. Цена фьючерса...,economics,цена барреля нефти brent превысила отметку долларов впервые июля года декабря тасс цена нефть марки brent бирже ice лондоне выросла цена фьючерса нефти brent поставкой феврале года составила барре...
2116,"МОСКВА, 13 декабря. /ТАСС/. ""Вильярреал"" крупно победил мадридский ""Атлетико"" в домашнем матче 15-го тура чемпионата Испании по футболу. Встреча завершилась со счетом 3:0.\n\nМячи забили Мануэль Т...",sport,москва декабря тасс вильярреал крупно победил мадридский атлетико домашнем матче тура чемпионата испании футболу встреча завершилась счетом мячи забили мануэль тригерос минута хонатан дос сантос р...
393,"Мэттью Херберт собирает странные звуки для нового трека Большой оригинал\nМэттью Херберт готовит очередной альбом, где ни в коем случает не изменит\nсебе. Релиз все также будет полон необычных зву...",culture,мэттью херберт собирает странные звуки нового трека большой оригинал мэттью херберт готовит очередной альбом коем случает изменит релиз также полон необычных звуков имеющих никакого отношения музыке
3127,"Восток-Медиа (vostokmedia.com) Дальневосточный фестиваль художественных\nремесел ""Живая нить времен"" пройдет в Хабаровске. В нем примут участие\nмастера художественных промыслов и декоративно-прик...",culture,восток медиа vostokmedia com дальневосточный фестиваль художественных ремесел живая нить времен пройдет хабаровске нем примут участие мастера художественных промыслов декоративно прикладного искус...
2963,"Эксперты автомобильной отрасли провели масштабное исследование отечественного авторынка с целью определить, какие марки и модели автомобилей с АКПП в России продавались лучше других в текущем году...",auto,эксперты автомобильной отрасли провели масштабное исследование отечественного авторынка целью определить какие марки модели автомобилей акпп россии продавались других текущем году самой популярной...
1433,"Сахалинский экс-губернатор Александр Хорошавин, а также его семья заявили о нарушении их конституционных прав из-за конфискации всего имущества.\n\nВ частности, как пишет «Коммерсантъ», господа Хо...",politics,сахалинский экс губернатор александр хорошавин также семья заявили нарушении конституционных прав конфискации имущества частности пишет коммерсантъ господа хорошавины недовольны положениями принят...
2076,"Sport Arena выделяет ключевые моменты матча на Стадио Олимпико, завершившегося минимальной победой хозяев.\n\nРома – Милан: как это было\n\nЛучано Спаллетти был честен и прав\n\nМатематически преи...",sport,sport arena выделяет ключевые моменты матча стадио олимпико завершившегося минимальной победой хозяев рома милан это лучано спаллетти честен прав математически преимущество белых черными сказал ра...
3079,Православное христианство- каталог ресурсов (hristianstvo.ru) В сентябре 2010\nгода Святейший Патриарх Кирилл посетит Камчатку Вопросы подготовки к поездке\nПредстоятеля Русской Православной Церкв...,social,православное христианство каталог рес

---

In [15]:
def print_top_words(model, feature_names, n_top_words=7):
    for topic_idx, topic in enumerate(model.components_):
        message = "Тема %d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

---

In [17]:
#df['text_clean']

In [ ]:
# tf features 
tf_vectorizer = CountVectorizer( max_df=0.95, min_df=2, max_features=n_features )
tf = tf_vectorizer.fit_transform(df['text_clean'])
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
# LDA - латентное размещение Дирихле
lda = LatentDirichletAllocation( n_components=n_components, max_iter=5, 
                                learning_method='online', learning_offset=50.,
                                random_state=0 ).fit(tf)
print('\nLDA:\n')
print_top_words(lda, tf_feature_names)

---

In [ ]:
# tf-idf features 
tfidf_vectorizer = TfidfVectorizer( max_df=0.95, min_df=2, max_features=n_features)

tfidf = tfidf_vectorizer.fit_transform(df['text_clean'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
# NMF (Frobenius norm) - неотрицательное матричное разложение
nmf = NMF( n_components=n_components, random_state=1,alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(Frobenius norm):\n')
print_top_words( nmf, tfidf_feature_names )

---

In [ ]:
# NMF (generalized Kullback-Leibler divergence)  
nmf = NMF( n_components=n_components, random_state=1, beta_loss='kullback-leibler', 
          solver='mu', max_iter=1000, alpha=.1, l1_ratio=.5 ).fit(tfidf)
print('\nNMF(generalized Kullback-Leibler divergence):\n')
print_top_words(nmf, tfidf_feature_names )